In [5]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [6]:
training = pd.DataFrame()
training = pd.read_excel("NCAA_Seed_Training_Set2.0.xlsx")
testing = pd.DataFrame()
testing = pd.read_csv("NCAA_Seed_Test_Set2.0.csv")

In [7]:
ind_vars1 = ["Q1 Win", "Q1 Loss", "Q2 Win", "Q2 Loss", "Q3 Win", "Q3 Loss", "Q4 Win", "Q4 Loss"]

training_filt = training.dropna(subset = ["NET Rank"])
testing_filt = testing.dropna(subset = ["NET Rank"])

In [8]:
lr = linear_model.LinearRegression()
seed_train = training_filt.dropna(subset=["Bid Type"])
X_reg1 = seed_train[ind_vars1]
y_reg =seed_train["Overall Seed"]

XTrain, XTest, yTrain, yTest = train_test_split(X_reg1, y_reg, random_state=0, test_size=0.2)
lr.fit(X=XTrain, y=yTrain)

yPred = lr.predict(XTest)
print('r2: ', round(metrics.r2_score(yTest, yPred), 3))

scores = cross_val_score(lr, X_reg1, y_reg, cv=5, scoring='r2')

# Print results
print("Cross-validated R² scores:", scores)
print("Mean Cross-validated R² score:", round(scores.mean(), 3))
print("Standard deviation:", round(scores.std(), 3))

r2:  0.895
Cross-validated R² scores: [0.74260545 0.91998045 0.85332793 0.84169073 0.84040361]
Mean Cross-validated R² score: 0.84
Standard deviation: 0.057


In [9]:
final_pred = testing
final_pred["Initial Seed"] = np.nan
filtered_pred = testing_filt[
    (testing_filt["Bid Type"] == "AL") |
    (testing_filt["Bid Type"] == "AQ")
]
X_reg1_test = filtered_pred[ind_vars1]
predictions = lr.predict(X_reg1_test)
final_pred.loc[filtered_pred.index, "Initial Seed"] = predictions
final_pred["Overall Seed"] = final_pred["Initial Seed"].round(0)
final_pred["Overall Seed"] = final_pred["Overall Seed"].clip(lower=1, upper=68)
final_pred["Overall Seed"] = final_pred["Overall Seed"].fillna(0).astype(int)
final_pred = final_pred[["RecordID", "Overall Seed"]]
final_pred.to_csv('predictions.csv', index=False)